In [ ]:
import tkinter as tk
from tkinter import font
from PIL import Image, ImageTk
import math
import time  # Import time module for measuring execution time

class GameTreeNode:
    def _init_(self, parent, current_number, move, player_type):
        self.parent = parent
        self.current_number = current_number
        self.move = move
        self.player_type = player_type
        self.children = []

    def add_child(self, child):
        self.children.append(child)

class GameTree:
    def _init_(self):
        self.root = None

    def insert_node(self, parent, current_number, move, player_type):
        new_node = GameTreeNode(parent, current_number, move, player_type)
        if parent is None:
            self.root = new_node
        else:
            parent.add_child(new_node)
        return new_node

class Game:
    def _init_(self, master):
        self.master = master
        self.master.title("Multiplication Game")
        self.master.geometry("700x850")
        self.master.configure(bg="#561C24")

        self.game_tree = GameTree()
        self.current_node = None

        self.starting_number = None
        self.current_number = None
        self.player_score = 0
        self.computer_score = 0
        self.first_player = None
        self.selected_ai = "alphabeta"
        self.move_history = []

        custom_font = font.Font(family="Helvetica", size=12, weight="bold")

        self.label = tk.Label(master, text="Choose a starting number (8-18):", bg="#E8D8C4", fg="#561C24", font=custom_font)
        self.label.grid(row=0, column=0, columnspan=3, pady=(20, 10))

        self.entry = tk.Entry(master, font=custom_font, bg="#ffffff", fg="#1e1e2f", borderwidth=2, relief="flat")
        self.entry.grid(row=1, column=0, columnspan=3, pady=(0, 10))

        self.first_player_label = tk.Label(master, text="Who should play first?", bg="#E8D8C4", fg="#561C24", font=custom_font)
        self.first_player_label.grid(row=2, column=0, columnspan=3, pady=(10, 5))

        self.player_first_button = tk.Button(master, text="Player First", bg="#A0522D", fg="white", font=custom_font, 
                                             command=lambda: self.set_first_player("player"), relief="flat", bd=0)
        self.player_first_button.grid(row=3, column=0, padx=10, pady=(0, 10))

        self.computer_first_button = tk.Button(master, text="Computer First", bg="#A0522D", fg="white", font=custom_font, 
                                               command=lambda: self.set_first_player("computer"), relief="flat", bd=0)
        self.computer_first_button.grid(row=3, column=2, padx=10, pady=(0, 10))

        self.ai_label = tk.Label(master, text="Choose AI Algorithm:", bg="#E8D8C4", fg="#561C24", font=custom_font)
        self.ai_label.grid(row=4, column=0, columnspan=3, pady=(10, 5))

        self.ai_choice = tk.StringVar(value="alphabeta")
        self.minimax_button = tk.Radiobutton(master, text="Minimax", variable=self.ai_choice, value="minimax", bg="#E8D8C4", font=custom_font)
        self.minimax_button.grid(row=5, column=0, pady=(0, 10))

        self.alphabeta_button = tk.Radiobutton(master, text="Alpha-Beta Pruning", variable=self.ai_choice, value="alphabeta", bg="#E8D8C4", font=custom_font)
        self.alphabeta_button.grid(row=5, column=2, pady=(0, 10))

        self.start_button = tk.Button(master, text="Start Game", bg="#50c878", fg="white", font=custom_font, 
                                      command=self.start_game, relief="flat", bd=0)
        self.start_button.grid(row=6, column=0, columnspan=3, pady=(10, 20))
        self.start_button.config(state=tk.DISABLED)

        self.result_label = tk.Label(master, text="", bg="white", fg="#561C24", font=custom_font, wraplength=350)
        self.result_label.grid(row=7, column=0, columnspan=3, pady=(10, 10))

        self.history_text = tk.Text(master, height=5, width=50, bg="white", fg="#561C24", font=custom_font, wrap=tk.WORD)
        self.history_text.grid(row=8, column=0, columnspan=3, pady=(0, 10))

        self.time_label = tk.Label(master, text="AI Decision Time: 0.0000 sec", bg="white", fg="#561C24", font=custom_font)
        self.time_label.grid(row=9, column=0, columnspan=3, pady=(10, 10))

    def set_first_player(self, player):
        self.first_player = player
        self.start_button.config(state=tk.NORMAL)

    def start_game(self):
        try:
            self.starting_number = int(self.entry.get())
            if 8 <= self.starting_number <= 18:
                self.current_number = self.starting_number
                self.result_label.config(text=f"Current number: {self.current_number}")
                self.selected_ai = self.ai_choice.get()

                if self.first_player == "computer":
                    self.master.after(1000, self.computer_turn)
        except ValueError:
            self.result_label.config(text="Invalid input. Please enter a valid integer.")

    def computer_turn(self):
        if self.current_number < 1200:
            best_score = -math.inf
            best_multiplier = 2

            start_time = time.time()  # Start measuring time

            for multiplier in [2, 3, 4]:
                new_number = self.current_number * multiplier
                if self.selected_ai == "alphabeta":
                    score = self.minimax(new_number, 3, -math.inf, math.inf, False)
                else:
                    score = self.minimax_no_prune(new_number, 3, False)

                if score > best_score:
                    best_score = score
                    best_multiplier = multiplier

            end_time = time.time()  # Stop measuring time
            time_taken = end_time - start_time

            self.current_number *= best_multiplier
            self.result_label.config(text=f"Computer ×{best_multiplier} → {self.current_number}")
            self.move_history.append(f"Computer ×{best_multiplier} → {self.current_number} (Time: {time_taken:.4f} sec)")
            self.update_history()

            self.time_label.config(text=f"AI Decision Time: {time_taken:.4f} sec")

    def minimax(self, number, depth, alpha, beta, maximizing_player):
        if depth == 0 or number >= 1200:
            return self.evaluate(number)
        if maximizing_player:
            max_eval = -math.inf
            for multiplier in [2, 3, 4]:
                new_number = number * multiplier
                eval = self.minimax(new_number, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = math.inf
            for multiplier in [2, 3, 4]:
                new_number = number * multiplier
                eval = self.minimax(new_number, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def minimax_no_prune(self, number, depth, maximizing_player):
        if depth == 0 or number >= 1200:
            return self.evaluate(number)
        return max(self.minimax_no_prune(number * multiplier, depth - 1, not maximizing_player) for multiplier in [2, 3, 4])

    def evaluate(self, number):
        return 0 if number >= 1200 else (1200 - number) / 1200

root = tk.Tk()
game = Game(root)
root.mainloop()